<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/final_lstm_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [ ]:
import pandas as pd

df = pd.read_csv('final_task2_data.csv')

print(df.head())


   Unnamed: 0                                               text  label
0           0  CMV: Fat Acceptance Tik Tokers and influencers...      0
1           1  CMV: It makes sense to compare apples and oranges      0
2           2  CMV: Dismissing an individual's experience bec...      0
3           3  CMV: If pro-anorexia groups, suicide promotion...      0
4           4  CMV: Asian Americans generally prospering in t...      0


In [ ]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',tokenizer_language='en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
df[['text']] = df[['text']]. astype(str)

<ipython-input-18-42dfdfbe4dfa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['text']] = df[['text']]. astype(str)


In [ ]:
df = df[['text','label']]

In [ ]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm',  include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

pt = [TEXT.preprocess(x) for x in df['text']]

TEXT.build_vocab(
    pt, max_size = 25000,
    vectors='glove.6B.50d', unk_init = torch.Tensor.normal_)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.33MB/s]                           
100%|█████████▉| 399999/400000 [00:11<00:00, 33790.37it/s]


In [ ]:
label_to_index= {label: index for index, label in enumerate(df['label'].unique())}

In [ ]:
df['label_new'] = df['label'].apply(lambda y: label_to_index[y])

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], fields)

In [ ]:
df_new=df[['text','label_new']]

In [ ]:
examples=[]
for i,row in df_new.iterrows():
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [ ]:
itrr=0
for i in examples:
  if type(i.label) != int:
    itrr+=1

In [ ]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [ ]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.70, 0.15, 0.15], stratified=True, strata_field='label')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cuda


In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim, padding_idx = pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, dropout = dropout, bidirectional = bidirectional)
        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim * n_layers, output_dim)
        #4. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        embedding = self.embedding(text)
        embedding_dropout = self.dropout(embedding)
        #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
        output, (hidden,cell) = self.lstm(embedding_dropout)
        #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(concat)     
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,720,611 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([30002, 50])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.3260,  2.3001,  0.0365,  ..., -1.4911,  0.6114,  0.9878],
        [ 0.0854, -0.6890,  0.6959,  ..., -0.3637,  0.2291,  0.6251],
        [ 0.7506,  0.1679, -0.3358,  ...,  1.3170, -0.5806, -1.2388]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.3260,  2.3001,  0.0365,  ..., -1.4911,  0.6114,  0.9878],
        [ 0.0854, -0.6890,  0.6959,  ..., -0.3637,  0.2291,  0.6251],
        [ 0.7506,  0.1679, -0.3358,  ...,  1.3170, -0.5806, -1.2388]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss()

# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        target = target.type(torch.LongTensor)

        # print(target)

        # print(predictions)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 15
path = F"/content/gdrive/My Drive/bilstm_model_task3.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 6s
	Train Loss: 0.517 | Train Acc: 84.94%
	 Val. Loss: 0.385 |  Val. Acc: 86.58%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss: 0.395 | Train Acc: 86.63%
	 Val. Loss: 0.344 |  Val. Acc: 86.58%
Epoch: 03 | Epoch Time: 0m 5s
	Train Loss: 0.340 | Train Acc: 87.46%
	 Val. Loss: 0.254 |  Val. Acc: 92.34%
Epoch: 04 | Epoch Time: 0m 5s
	Train Loss: 0.315 | Train Acc: 88.16%
	 Val. Loss: 0.321 |  Val. Acc: 86.58%
Epoch: 05 | Epoch Time: 0m 5s
	Train Loss: 0.283 | Train Acc: 89.84%
	 Val. Loss: 0.226 |  Val. Acc: 91.85%
Epoch: 06 | Epoch Time: 0m 5s
	Train Loss: 0.232 | Train Acc: 92.42%
	 Val. Loss: 0.205 |  Val. Acc: 94.06%
Epoch: 07 | Epoch Time: 0m 5s
	Train Loss: 0.205 | Train Acc: 93.77%
	 Val. Loss: 0.197 |  Val. Acc: 94.17%
Epoch: 08 | Epoch Time: 0m 5s
	Train Loss: 0.192 | Train Acc: 94.42%
	 Val. Loss: 0.200 |  Val. Acc: 93.96%
Epoch: 09 | Epoch Time: 0m 5s
	Train Loss: 0.185 | Train Acc: 94.37%
	 Val. Loss: 0.215 |  Val. Acc: 94.33%
Epoch: 10 | Epoch Time: 0m 4

In [ ]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.213 | Test Acc: 94.15%


In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# load the trained model
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
y_true = []
y_pred = []

model.load_state_dict(torch.load("/content/gdrive/My Drive/bilstm_model_task3.pt"))
model=model.to(device)

for batch in test_iter:
    text, labels = batch
    text,text_length=text
    predictions = model(text,text_length).squeeze(1)
    y_true.extend(labels.tolist())
    y_pred.extend(predictions.argmax(dim=1).tolist())  # convert to list of labels

# calculate the evaluation metrics using sklearn's functions
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)
import pandas as pd


Accuracy: 0.9343598055105349
Precision: 0.8682866176192692
Recall: 0.8329932876269788
F1 score: 0.8493368524344049


In [ ]:
df_reddit = pd.read_csv('final_reddit_text_curie_output.csv')

In [ ]:
df_reddit.head()

,Unnamed: 0,Reddit Prompts,Reddit Gpt3 Response,Model Name
0,0,CMV: Fat Acceptance Tik Tokers and influencers...,There is no denying that there is a dangerous ...,GPT-3
1,1,CMV: It makes sense to compare apples and oranges,One common way to compare two things is to com...,GPT-3
2,2,CMV: Dismissing an individual's experience bec...,NaN,GPT-3
3,3,"CMV: If pro-anorexia groups, suicide promotion...",due to the vulnerability of the users.,GPT-3
4,4,CMV: Asian Americans generally prospering in t...,There is no correlation between success of one...,GPT-3


In [ ]:
import pandas as pd
new_names = {'Reddit Prompts':'Title', 'Reddit Gpt3 Response': 'Body'}
df_reddit = df_reddit.rename(columns=new_names)

print(df_reddit)


     Unnamed: 0                                              Title  \
0             0  CMV: Fat Acceptance Tik Tokers and influencers...   
1             1  CMV: It makes sense to compare apples and oranges   
2             2  CMV: Dismissing an individual's experience bec...   
3             3  CMV: If pro-anorexia groups, suicide promotion...   
4             4  CMV: Asian Americans generally prospering in t...   
..          ...                                                ...   
555         555  How massive of a red flag is it that I (27M) h...   
556         556  Should I (25F) keep talking to my ex hookup's ...   
557         557  My girlfriend [25F] and I [26M] have almost no...   
558         558  I(15M) said it was my dream to make music with...   
559         559  I (19m) flirting with two people 19f, 20(f) wh...   

                                                  Body Model Name  
0    There is no denying that there is a dangerous ...      GPT-3  
1    One common way to 

In [ ]:
df_reddit['text'] = df_reddit['Title']+df_reddit['Body']

In [ ]:
df_reddit.head()

,text,label
0,CMV: Fat Acceptance Tik Tokers and influencers...,9
1,CMV: It makes sense to compare apples and oran...,9
2,NaN,9
3,"CMV: If pro-anorexia groups, suicide promotion...",9
4,CMV: Asian Americans generally prospering in t...,9
